## <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Generating Automated responses to Customer Reviews using GenAI


### License

In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### Notebook Overview

- This notebook will use GenAI to create 5 potential automated responses based upon each customer review. The responses can then by used to reply to the customer via the web application.
    - Example Responses:
        - "We are sorry for the bad service you received.  We always strive for execellent service and have let the coffee truck know.  We are attaching a coupon for a free coffee."
        - "Thank you for the review you left of social media.  We depend upon our reviews to help us deliver greate customer statisfaction."


- Notebook Logic:
    1. Gather the reviews that need a GenAI response created.

    2. Create our LLM prompt:
        - The prompt will contain the review text.
        - The prompt will ask for 5 responses to be created.
        - The prompt will ask the LLM to address any concerns within the review text.
        - The prompt will provide example JSON results so the LLM knows the output format to generate the results.

    3. Execute our LLM prompt using BigFrames

    4. Parse the JSON from the LLM and construct an UPDATE SQL statement so we can update the cutomer review table.
        - We check for valid JSON in case the LLM created incomplete or invalid JSON
        
    5. Execute the SQL statement against BigQuery.

## Initialize Python

In [ ]:
import pandas as pd
import json
import bigframes.pandas as bf
#from bigframesllm import BigFramesLLM
from bigframes.ml.llm import GeminiTextGenerator

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

In [ ]:
PROJECT_ID = "${project_id}"
REGION = "us"
DATASET_ID = "${bigquery_data_beans_curated_dataset}"
CONNECTION_NAME = "vertex-ai"

connection = f"{PROJECT_ID}.{REGION}.{CONNECTION_NAME}"

In [ ]:
# bf.reset_session() # if you need to change the region
bf.options.bigquery.project = PROJECT_ID
bf.options.bigquery.location = REGION

In [ ]:
session = bf.get_global_session()

llm_model = GeminiTextGenerator(session=session, connection_name=connection)

## Supporting Functions

In [ ]:
def PrettyPrintJson(json_string):
  try:
    json_object = json.loads(json_string)
    json_formatted_str = json.dumps(json_object, indent=2)
    print(json_formatted_str)
    return json.dumps(json_object)
  except Exception as error:
    print("PrettyPrintJson: An error occurred:", error)
    return None

In [ ]:
def LLM(prompt, isOutputJson, max_output_tokens=1024, temperature=0, top_p=0, top_k=1):
  print()
  print("Prompt: ", prompt)
  print()
  df_prompt = pd.DataFrame(
          {
              "prompt": [prompt],
          })
  bf_df_prompt = bf.read_pandas(df_prompt)
  prediction = llm_model.predict(bf_df_prompt,
                                 max_output_tokens=max_output_tokens,
                                 temperature=temperature, # 0 to 1 (1 random)
                                 top_p=top_p, # 0 to 1 (1 random)
                                 top_k=top_k, # (1 to 40 random)
                                 ).to_pandas()
  try:
    # Remove common LLM output mistakes
    result = prediction['ml_generate_text_llm_result'][0]

    result = result.replace("```json\n","")
    result = result.replace("```JSON\n","")
    result = result.replace("```json","")
    result = result.replace("```JSON","")
    result = result.replace("```sql\n","")
    result = result.replace("```SQL\n","")
    result = result.replace("```sql","")
    result = result.replace("```SQL","")
    result = result.replace("```","")

    if isOutputJson:
      result = result.replace("\n"," ")
      json_string = PrettyPrintJson(result)
      json_string = json_string.replace("'","\\'")
      json_string = json_string.strip()
      return json_string
    else:
      if "INSERT INTO" in result:
        # do nothing
        print("")
      else:
        result = result.replace("'","\\'")
      result = result.strip()
      return result

  except:
    print("Error (raw): ", prediction['ml_generate_text_llm_result'][0])
    print("Error (result): ", result)


In [ ]:
# Runs a query against BigQuery and waits for it to complete
def RunQuery(sql):
  import time

  #return True # return early for now

  job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
  query_job = client.query(sql, job_config=job_config)

  # Check on the progress by getting the job's updated state.
  query_job = client.get_job(
      query_job.job_id, location=query_job.location
  )
  print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

  while query_job.state != "DONE":
    time.sleep(5)
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
        )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

  if query_job.error_result == None:
    return True
  else:
    return False

## Generate Responses using GenAI

In [ ]:
sql = """SELECT customer_review_id,
                review_text
          FROM `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review`
         WHERE gen_ai_reponse IS NULL
        ORDER BY customer_review_id DESC"""

df_process = client.query(sql).to_dataframe()

for row in df_process.itertuples():
  customer_review_id = row.customer_review_id
  review_text = row.review_text

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      prompt=f"""
      You need to return JSON in the below "JSON format".
      Generate responses to the below customer review who purchased coffee and return the results the below json format.
      The review can be positive, negative, or neutral.
      Provide a variety of responses, including thanking customers for positive reviews, addressing concerns in negative reviews, and engaging with neutral reviews.
      Please generate at least 5 different responses.
      Do not include newline characters in the response.

      JSON format: [ "value" ]
      Sample JSON Response: [ "response 1", "response 2", "response 3", "response 4", "response 5" ]

      Review: {review_text}"""

      json_result = LLM(prompt, True, max_output_tokens=1024, temperature=0, top_p=0, top_k=1)
      print(f"json_result: {json_result}")

      if json_result == None:
        llm_valid_execution = True
        llm_valid_execution = False
      else:
        sql = f"""UPDATE `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review`
                    SET gen_ai_reponse = '{json_result}'
                  WHERE customer_review_id = {customer_review_id}
        """

        print(f"sql: {sql}")

        llm_valid_execution = RunQuery(sql)
    except Exception as error:
      print("An error occurred:", error)